# Voting (1)

## 1. 패키지 참조

In [20]:
from pandas import DataFrame, read_excel
from sklearn.ensemble import VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

## 2. 데이터 가져오기

In [5]:
origin = read_excel('https://data.hossam.kr/G02/breast_cancer.xlsx')
origin.info()
origin.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 569 entries, 0 to 568
Data columns (total 31 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   mean radius              569 non-null    float64
 1   mean texture             569 non-null    float64
 2   mean perimeter           569 non-null    float64
 3   mean area                569 non-null    float64
 4   mean smoothness          569 non-null    float64
 5   mean compactness         569 non-null    float64
 6   mean concavity           569 non-null    float64
 7   mean concave points      569 non-null    float64
 8   mean symmetry            569 non-null    float64
 9   mean fractal dimension   569 non-null    float64
 10  radius error             569 non-null    float64
 11  texture error            569 non-null    float64
 12  perimeter error          569 non-null    float64
 13  area error               569 non-null    float64
 14  smoothness error         5

,mean radius,mean texture,mean perimeter,mean area,mean smoothness,mean compactness,mean concavity,mean concave points,mean symmetry,mean fractal dimension,...,worst texture,worst perimeter,worst area,worst smoothness,worst compactness,worst concavity,worst concave points,worst symmetry,worst fractal dimension,target
0,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,0.07871,...,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,0
1,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,0.05667,...,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,0
2,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,0.05999,...,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758,0
3,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,0.09744,...,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,0
4,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,0.05883,...,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,0


## 3. 데이터 전처리
### 1) 독립/종속 변수 분리

In [6]:
x = origin.drop('target', axis = 1)
y = origin['target']
x.shape, y.shape

((569, 30), (569,))

### 2) 데이터 표준화
- `StandardScaler()`

In [9]:
scaler = StandardScaler() # 표준화 객체 만들기
std_x = scaler.fit_transform(x)
std_x[:5] # 표준화하면 2차열 배열로 바뀐다 (필드 이름들 다 빠진다)

array([[ 1.09706398e+00, -2.07333501e+00,  1.26993369e+00,
         9.84374905e-01,  1.56846633e+00,  3.28351467e+00,
         2.65287398e+00,  2.53247522e+00,  2.21751501e+00,
         2.25574689e+00,  2.48973393e+00, -5.65265059e-01,
         2.83303087e+00,  2.48757756e+00, -2.14001647e-01,
         1.31686157e+00,  7.24026158e-01,  6.60819941e-01,
         1.14875667e+00,  9.07083081e-01,  1.88668963e+00,
        -1.35929347e+00,  2.30360062e+00,  2.00123749e+00,
         1.30768627e+00,  2.61666502e+00,  2.10952635e+00,
         2.29607613e+00,  2.75062224e+00,  1.93701461e+00],
       [ 1.82982061e+00, -3.53632408e-01,  1.68595471e+00,
         1.90870825e+00, -8.26962447e-01, -4.87071673e-01,
        -2.38458552e-02,  5.48144156e-01,  1.39236330e-03,
        -8.68652457e-01,  4.99254601e-01, -8.76243603e-01,
         2.63326966e-01,  7.42401948e-01, -6.05350847e-01,
        -6.92926270e-01, -4.40780058e-01,  2.60162067e-01,
        -8.05450380e-01, -9.94437403e-02,  1.80592744e+

### 3) 훈련/검증 데이터 분할
- `train_test_split()`

In [12]:
x_train, x_test, y_train, y_test = train_test_split(std_x, 
                                                    y, 
                                                    test_size = 0.3,
                                                    random_state=777)
x_train.shape, x_test.shape, y_train.shape, y_test.shape

((398, 30), (171, 30), (398,), (171,))

## 4. Simple Model
### 1) 개별 모델 정의

In [22]:
# 각 객체에 각 모델을 정의
lr = LogisticRegression()
knn = KNeighborsClassifier()
dt = DecisionTreeClassifier()

### 2) 개별 모델을 앙상블 모델로 결합

In [25]:
# 학습 객체 만들기
vo = VotingClassifier(
    # 결합할 알고리즘의 리스트 (이름, 알고리즘객체) - 이름은 개발자가 마음대로 지정
    estimators=[('LR', lr), ('KNN', knn), ('DTREE', dt)],
    # hard: 다수결, soft: 확률의 평균 # 둘 중 뭐가 성능이 더 잘 나오는지 알아봐야 한다
    voting='soft')

# 학습 수행 (훈련 데이터로 해야 됨)
vo.fit(x_train, y_train)

# 예측 수행 (검증 데이터로 해야 됨)
y_pred = vo.predict(x_test)

# 검증 데이터에 대한 정확도
score = accuracy_score(y_test, y_pred)
print(f'Voting 분류기 정확도: {score:.4f}')

Voting 분류기 정확도: 0.9708


### 3) (공부용) 개별 모델과의 결과 비교

In [26]:
classifier = [lr, knn, dt]
for c in classifier:
    c.fit(x_train, y_train)
    y_pred = c.predict(x_test)
    score = accuracy_score(y_test, y_pred)
    print(f'{c.__class__.__name__} 정확도: {score:.4f}')

LogisticRegression 정확도: 0.9766
KNeighborsClassifier 정확도: 0.9708
DecisionTreeClassifier 정확도: 0.9181


> 대부분의 예제들에서는 Voting의 성능이 좀 더 높게 나오지만, 항상 그렇다고 할 수 없다.
>
> 여러번 실행한다면 Voting 분류기의 성능이 더 높을 때도, 더 낮을 때도 발생하므로 여러번 실험을 수행헤서 성능이 가장 좋은 조합/단일 모델을 만들어야 된다.